In [158]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statistics as st

def get_df(day):
    file_name = f"../data/prices_round_3_day_{day}.csv"
    return pd.read_csv(file_name, sep=';')


def get_trades_df(day):
    file_name = f"../data/trades_round_3_day_{day}.csv"
    return pd.read_csv(file_name, sep=';')


def get_product(df, product):
    return df[df['product'] == product].copy()



def get_first_two_dfs():
    first_df = get_df(1)
    second_df = get_df(2)
    second_df['timestamp'] = second_df['timestamp'] + 1000000
    return pd.concat([first_df, second_df])

In [159]:

df = get_first_two_dfs()

In [160]:
df_volcanic = get_product(df, 'VOLCANIC_ROCK')
df_volcanic_voucher_9500 = get_product(df, "VOLCANIC_ROCK_VOUCHER_9500")

In [161]:
print(df_volcanic_voucher_9500.head(5))
df_volcanic.head(5)

    day  timestamp                     product  bid_price_1  bid_volume_1  \
3     1          0  VOLCANIC_ROCK_VOUCHER_9500       1015.0          12.0   
24    1        100  VOLCANIC_ROCK_VOUCHER_9500       1015.0          13.0   
28    1        200  VOLCANIC_ROCK_VOUCHER_9500       1015.0          16.0   
48    1        300  VOLCANIC_ROCK_VOUCHER_9500       1017.0          14.0   
67    1        400  VOLCANIC_ROCK_VOUCHER_9500       1019.0          18.0   

    bid_price_2  bid_volume_2  bid_price_3  bid_volume_3  ask_price_1  \
3           NaN           NaN          NaN           NaN         1017   
24          NaN           NaN          NaN           NaN         1016   
28          NaN           NaN          NaN           NaN         1016   
48          NaN           NaN          NaN           NaN         1018   
67          NaN           NaN          NaN           NaN         1020   

    ask_volume_1  ask_price_2  ask_volume_2  ask_price_3  ask_volume_3  \
3             12        

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,1,0,VOLCANIC_ROCK,10515.0,143.0,10514.0,57.0,NaN,NaN,10517,200,NaN,NaN,NaN,NaN,10516.0,0.0
27,1,100,VOLCANIC_ROCK,10514.0,104.0,10513.0,44.0,NaN,NaN,10516,104,10517.0,44.0,NaN,NaN,10515.0,0.0
33,1,200,VOLCANIC_ROCK,10514.0,133.0,10513.0,62.0,NaN,NaN,10516,133,10517.0,62.0,NaN,NaN,10515.0,0.0
43,1,300,VOLCANIC_ROCK,10516.0,200.0,NaN,NaN,NaN,NaN,10518,149,10519.0,51.0,NaN,NaN,10517.0,0.0
58,1,400,VOLCANIC_ROCK,10519.0,108.0,10518.0,63.0,NaN,NaN,10520,103,10521.0,63.0,NaN,NaN,10519.5,0.0


In [162]:
df_volcanic_voucher_9500['mid_price'].mean()

771.296125

In [163]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_volcanic['timestamp'],
    y=df_volcanic['mid_price'],
    name='Volcanic Rock Mid Price',
    yaxis='y1',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=df_volcanic_voucher_9500['timestamp'],
    y=df_volcanic_voucher_9500['mid_price'],
    name='Voucher Call Mid Price',
    yaxis='y2',
    line=dict(color='orange')
))

fig.update_layout(
    title='Volcanic Rock & Call Option (9500) Prices',
    xaxis=dict(title='Timestamp'),
    yaxis=dict(title='Volcanic Rock Mid Price', side='left'),
    yaxis2=dict(title='Call Option Mid Price', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=600
)

fig.show()

In [164]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    call_price = (spot * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_price

def black_scholes_put(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    put_price = (strike * norm.cdf(-d2) - spot * norm.cdf(-d1))
    return put_price

def delta(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.cdf(d1)

def gamma(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1)/(spot * volatility * np.sqrt(time_to_expiry))

def vega(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1) * (spot * np.sqrt(time_to_expiry)) / 100

def implied_volatility(call_price, spot, strike, time_to_expiry):
    # Define the equation where the root is the implied volatility
    def equation(volatility):
        estimated_price = black_scholes_call(spot, strike, time_to_expiry, volatility)
        return estimated_price - call_price

    # Using Brent's method to find the root of the equation
    implied_vol = brentq(equation, 1e-10, 3.0, xtol=1e-10)
    return implied_vol

def safe_implied_volatility(call_price, spot, strike, time_to_expiry):
    try:
        return implied_volatility(call_price, spot, strike, time_to_expiry)
    except ValueError:
        return np.nan



def realized_vol(df_volcanic_voucher_9500, window, step_size):
    df_volcanic_voucher_9500[f'log_return_{step_size}'] = np.log(df_volcanic_voucher_9500['mid_price_volcanic_rock'].to_numpy()/df_volcanic_voucher_9500['mid_price_volcanic_rock'].shift(step_size).to_numpy())
    dt = step_size / 250 / 10000 
    df_volcanic_voucher_9500[f'realized_vol_{step_size}'] = df_volcanic_voucher_9500[f'log_return_{step_size}'].rolling(window=window).apply(lambda x: np.mean(x[::step_size]**2) / dt)
    df_volcanic_voucher_9500[f'realized_vol_{step_size}'] = np.sqrt(df_volcanic_voucher_9500[f'realized_vol_{step_size}'].to_numpy())
    return df_volcanic_voucher_9500




In [165]:
df_volcanic_rock = df_volcanic[['timestamp', 'mid_price']].copy()
df_volcanic_rock.rename(columns={'mid_price': 'mid_price_volcanic_rock'}, inplace=True)
df_volcanic_voucher_9500 = df_volcanic_voucher_9500.merge(df_volcanic_rock, on='timestamp', how='left')


In [166]:
# Parametry opcji
strike_price = 9500
time_to_expiry = 0.019  # ≈ 5 dni handlowych

df_volcanic_voucher_9500['implied_vol'] = df_volcanic_voucher_9500.apply(
    lambda row: safe_implied_volatility(
        row['mid_price'], 
        row['mid_price_volcanic_rock'], 
        strike_price, 
        time_to_expiry
    ), axis=1
)


df_volcanic_voucher_9500['delta'] = df_volcanic_voucher_9500.apply(
    lambda row: delta(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)

df_volcanic_voucher_9500['gamma'] = df_volcanic_voucher_9500.apply(
    lambda row: gamma(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)

df_volcanic_voucher_9500['vega'] = df_volcanic_voucher_9500.apply(
    lambda row: vega(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)

In [167]:
fig_prices = go.Figure()
fig_prices.add_trace(go.Scatter(x=df_volcanic['timestamp'], y=df_volcanic['mid_price'], name='Volcanic Rock Mid Price', yaxis='y1', line=dict(color='blue')))
fig_prices.add_trace(go.Scatter(x=df_volcanic_voucher_9500['timestamp'], y=df_volcanic_voucher_9500['mid_price'], name='Voucher Call Mid Price', yaxis='y2', line=dict(color='orange')))
fig_prices.update_layout(
    title='Volcanic Rock & Call Option (9500) Prices',
    xaxis=dict(title='Timestamp'),
    yaxis=dict(title='Volcanic Rock Mid Price', side='left'),
    yaxis2=dict(title='Call Option Mid Price', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=600
)
fig_prices.show()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,...,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,mid_price_volcanic_rock,implied_vol,delta,gamma,vega
0,1,0,VOLCANIC_ROCK_VOUCHER_9500,1015.0,12.0,NaN,NaN,NaN,NaN,1017,...,NaN,NaN,NaN,1016.0,0.0,10516.0,1.000000e-10,1.000000,0.000000,0.000000
1,1,100,VOLCANIC_ROCK_VOUCHER_9500,1015.0,13.0,NaN,NaN,NaN,NaN,1016,...,NaN,NaN,NaN,1015.5,0.0,10515.0,2.792566e-01,0.996051,0.000029,0.169755
2,1,200,VOLCANIC_ROCK_VOUCHER_9500,1015.0,16.0,NaN,NaN,NaN,NaN,1016,...,NaN,NaN,NaN,1015.5,0.0,10515.0,2.792566e-01,0.996051,0.000029,0.169755
3,1,300,VOLCANIC_ROCK_VOUCHER_9500,1017.0,14.0,NaN,NaN,NaN,NaN,1018,...,NaN,NaN,NaN,1017.5,0.0,10517.0,2.797218e-01,0.996058,0.000029,0.169526
4,1,400,VOLCANIC_ROCK_VOUCHER_9500,1019.0,18.0,NaN,NaN,NaN,NaN,1020,...,16.0,NaN,NaN,1019.5,0.0,10519.5,1.000000e-10,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,1999500,VOLCANIC_ROCK_VOUCHER_9500,667.0,5.0,NaN,NaN,NaN,NaN,669,...,NaN,NaN,NaN,668.0,0.0,10168.0,1.000000e-10,1.000000,0.000000,0.000000
19996,2,1999600,VOLCANIC_ROCK_VOUCHER_9500,667.0,14.0,NaN,NaN,NaN,NaN,668,...,NaN,NaN,NaN,667.5,0.0,10167.0,1.954193e-01,0.994338,0.000059,0.226336
19997,2,1999700,VOLCANIC_ROCK_VOUCHER_9500,667.0,13.0,NaN,NaN,NaN,NaN,668,...,NaN,NaN,NaN,667.5,0.0,10167.0,1.954193e-01,0.994338,0.000059,0.226336
19998,2,1999800,VOLCANIC_ROCK_VOUCHER_9500,667.0,20.0,NaN,NaN,NaN,NaN,668,...,NaN,NaN,NaN,667.5,0.0,10167.5,1.000000e-10,1.000000,0.000000,0.000000


In [169]:
df_volcanic_voucher_9500['implied_vol'].mean()

0.12790445887824495